# Technology Review
**Topic:** Comparing Scikit-learn to statsmodels for Logistic Regression\
**Group:** Live Lite\
**Group Members:** Parvati Jayakumar, Ted Liu, Saikripa Mohan, Manasa Shivappa

## 1. Import Libraries, Add Dependencies, and Define all Functions

In [1]:
import pandas as pd
import time
import joblib
import os

import statsmodels.api as sm

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression as LogisticRegressionSklearn
from sklearn.metrics import accuracy_score, classification_report

In [2]:
dump_models_path = ['Demo-results', 'Demo-results/scikit-learn_models', 'Demo-results/statsmodels_models']
for path in dump_models_path:
    if not os.path.exists(path):
        os.makedirs(path)

In [3]:
# Function to apply One-hot encoding
def apply_one_hot_encoding():
    categorical_features = ['Ethnicity', 'Depressed', 'StressEating', 'Smoke', 'ActivityLevel', 'Health', 'Diet', 'Gender']
    preprocessor = ColumnTransformer(transformers=[('cat', OneHotEncoder(), categorical_features)],
                                     remainder='passthrough')
    return preprocessor

In [4]:
# Function to fit and save a scikit-learn model
def train_scikitlearn_model(X_train, y_train, filename, binary_output_flag=False):
    # Fit the Scikit-learn model
    start_time_sklearn = time.time()
    
    if binary_output_flag:
        model_sklearn = LogisticRegressionSklearn()
        
    else:
        preprocessor = apply_one_hot_encoding()
        model_sklearn = Pipeline([('preprocessor', preprocessor),
                                  ('classifier', LogisticRegressionSklearn(max_iter=2000))])
        
    model_sklearn.fit(X_train, y_train)
    
    end_time_sklearn = time.time()
    
    # Save the trained model
    joblib.dump(model_sklearn, filename)
    
    # Return the time it took to train the model
    return start_time_sklearn, end_time_sklearn

In [5]:
# Function to fit and save a statsmodels model
def train_statsmodels(X_train, y_train, filename, binary_output_flag=False):
    # Train the statsmodels model
    if binary_output_flag:
        X_train_sm = sm.add_constant(X_train)  # statsmodels requires us to add a constant column for the intercept
    else:
        X_train_sm = X_train
    start_time_statsmodels = time.time()
    model_statsmodels = sm.Logit(y_train, X_train_sm).fit(disp=0)  # disp=0 if we want to suppress the fit summary output
    end_time_statsmodels = time.time()
    
    # Save the trained model
    joblib.dump(model_statsmodels, filename)
    
    # Return the time it took to train the model
    return start_time_statsmodels, end_time_statsmodels

In [6]:
# Function to evaluate the trained scikit learn model
def evaluate_scikitlearn_model(X_test, y_test, start_time, end_time, filename):
    
    # Load the trained model
    model_sklearn = joblib.load(filename)
    
    # Predict the results on the test set
    y_pred_sklearn = model_sklearn.predict(X_test)
    
    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    
    print("Model Training time: ", round(elapsed_time,5), "seconds")
    print("Model Accuracy: ", round(accuracy_score(y_test, y_pred_sklearn),5)*100, "%")
    print("Classification Report: \n", classification_report(y_test, y_pred_sklearn))

In [7]:
# Function to evaluate the trained statsmodels model
def evaluate_statsmodels_model(X_test, y_test, start_time, end_time, filename, binary_output_flag=False):
    model_statsmodels = joblib.load(filename)
    
    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    
    print("Model Training time: ", round(elapsed_time,3), "seconds")
    
    if binary_output_flag: # Convert probabilities to binary output
        X_test_sm = sm.add_constant(X_test)
        y_pred_statsmodels = model_statsmodels.predict(X_test_sm) > 0.5 
        print("Model Accuracy: ", round(accuracy_score(y_test, y_pred_statsmodels),3)*100, "%")
        print("Classification Report: \n", classification_report(y_test, y_pred_statsmodels))
    else:
        print("\n", model_statsmodels.summary())

## 2. Data preparation

In [8]:
# Generate a random classification dataset with 20 features and 2 categories (binary classification).
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)

In [9]:
# Split the dataset into training and testing sets
X_train_random, X_test_random, y_train_random, y_test_random = train_test_split(X, y, test_size=0.2, random_state=42)

## 3. Training

### 3.1 Using 'Scikit-learn'

In [10]:
# Fit a logistic regression model using scikit-learn and time it
start_time_sklearn_random, end_time_sklearn_random = train_scikitlearn_model(X_train_random, y_train_random, 'Demo-results/scikit-learn_models/random-model.demo', True)

### 3.2 Using 'statsmodels'

In [11]:
# Fit a logistic regression model using statsmodels and time it
start_time_statsmodels_random, end_time_statsmodels_random = train_statsmodels(X_train_random, y_train_random, 'Demo-results/statsmodels_models/random-model.demo', True)

## 4. Evaluate the models

### 4.1 Using 'Scikit-learn'

In [12]:
# Evaluate the scikit-learn model and print the results
evaluate_scikitlearn_model(X_test_random, y_test_random, start_time_sklearn_random, end_time_sklearn_random,'Demo-results/scikit-learn_models/random-model.demo')

Model Training time:  0.01925 seconds
Model Accuracy:  85.5 %
Classification Report: 
               precision    recall  f1-score   support

           0       0.80      0.91      0.85        93
           1       0.91      0.80      0.86       107

    accuracy                           0.85       200
   macro avg       0.86      0.86      0.85       200
weighted avg       0.86      0.85      0.86       200



### 4.2 Using 'statsmodels'

In [13]:
# Evaluate the statsmodels model and print the results
evaluate_statsmodels_model(X_test_random, y_test_random, start_time_statsmodels_random, end_time_statsmodels_random,'Demo-results/statsmodels_models/random-model.demo', True)

Model Training time:  0.04 seconds
Model Accuracy:  85.5 %
Classification Report: 
               precision    recall  f1-score   support

           0       0.80      0.91      0.85        93
           1       0.91      0.80      0.86       107

    accuracy                           0.85       200
   macro avg       0.86      0.86      0.85       200
weighted avg       0.86      0.85      0.86       200



## 5. Results on our project dataset

### 5.1 Data Preparation

In [14]:
# Read the dataset
df = pd.read_csv('Obesitysample.csv')

In [15]:
# Define model inputs and outputs
X = df.drop(['ID', 'IsObese'], axis=1)
y = df['IsObese']

# Split the dataset to train-test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 5.2 Training

#### 5.2.1 Using 'scikit-learn'

In [16]:
# Fit a logistic regression model using scikit-learn and time it
start_time_sklearn, end_time_sklearn = train_scikitlearn_model(X_train, y_train, 'Demo-results/scikit-learn_models/Obesity-risk-model.demo')

#### 5.2.2 Using 'statsmodels'

In [17]:
# Fit a logistic regression model using statsmodels and time it
start_time_statsmodels, end_time_statsmodels = train_statsmodels(X_train, y_train, 'Demo-results/statsmodels_models/Obesity-risk-model.demo')

### 5.3 Evaluate the model

#### 5.3.1 Using 'scikit-learn'

In [18]:
# Evaluate the scikit-learn model and print the results
evaluate_scikitlearn_model(X_test, y_test, start_time_sklearn, end_time_sklearn,'Demo-results/scikit-learn_models/Obesity-risk-model.demo')

Model Training time:  0.68643 seconds
Model Accuracy:  64.846 %
Classification Report: 
               precision    recall  f1-score   support

           0       0.68      0.82      0.74       723
           1       0.56      0.38      0.45       449

    accuracy                           0.65      1172
   macro avg       0.62      0.60      0.60      1172
weighted avg       0.63      0.65      0.63      1172



#### 5.3.2 Using 'statsmodels'

In [19]:
# Evaluate the statsmodels model and print the results
evaluate_statsmodels_model(X_test, y_test, start_time_statsmodels, end_time_statsmodels,'Demo-results/statsmodels_models/Obesity-risk-model.demo')

Model Training time:  0.035 seconds

                            Logit Regression Results                           
Dep. Variable:                IsObese   No. Observations:                 4684
Model:                          Logit   Df Residuals:                     4674
Method:                           MLE   Df Model:                            9
Date:                Fri, 09 Feb 2024   Pseudo R-squ.:                 0.04400
Time:                        18:24:01   Log-Likelihood:                -2962.2
converged:                       True   LL-Null:                       -3098.5
Covariance Type:            nonrobust   LLR p-value:                 1.616e-53
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Age              -0.0058      0.002     -3.563      0.000      -0.009      -0.003
Ethnicity        -0.1446      0.018     -8.124      0.000      -0.180      -0.110
De

### 5.4 Predict Obesity Risk

In [20]:
# Define the new data
predict_df = pd.DataFrame({'Age': [80],
                        'Ethnicity': [4],
                        'Depressed': [0],
                        'StressEating': [1],
                        'Smoke': [0],
                        'Sleephours': [2],
                        'ActivityLevel': [2],
                        'Health': [1],
                        'Diet': [4],
                        'Gender': [1]})

#### 5.4.1 Using 'scikit-learn'

In [21]:
model_scikitlearn = joblib.load('Demo-results/scikit-learn_models/Obesity-risk-model.demo')
scikitlearn_new_prediction = model_scikitlearn.predict_proba(predict_df)[:, 1]  # Probability of class 1 (obese)
Obesityrisk = round(scikitlearn_new_prediction[0] * 100, 2)
print(f'Predicted Obesity Risk: {Obesityrisk} %')

Predicted Obesity Risk: 43.99 %


#### 5.4.2 Using 'statsmodels'

In [22]:
model_statsmodels = joblib.load('Demo-results/statsmodels_models/Obesity-risk-model.demo')
statsmodels_new_prediction = model_statsmodels.predict(predict_df)
Obesityrisk = round(statsmodels_new_prediction[0] * 100, 2)
print(f'Predicted Obesity Risk: {Obesityrisk} %')

Predicted Obesity Risk: 42.88 %
